<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=4686d6748ecfbe8050edccc26af8c761b12f16a0b5f007f4a10a6af2e92e4a66
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-17 11:48:08
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  66.76 K
Current:  1.40 C
-------------------
Today PnL: -50.30 K (-0.36%)
Current PnL: -27.26 L (-17.61%)
CY Booked + Current PnL: -12.97 L (-8.38%)
-------------------
Total profit:  1.33 L
Total loss:  -28.59 L
-------------------
Total Booked + Current PnL: 14.01 L (10.96%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.22%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.32 C
Est FTT PnL: 91.89 L (65.73%)
Deployed:  1.28 C
Current:  1.40 C
CAGR/XIRR %: 4.97%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,-0.50,-18.49,22.80,0.09,18729.0,-18633.0,82144.0,90.61,33.0,M-SC,2.47,253.0,-0.99,0.59,7.92,OX40N,NTT,DURABLES
82,VOLTAS,1530.00,-0.57,8.45,10.37,19.69,21564.0,16203.0,207945.0,-2.18,56.0,X-MC,2.55,78.0,0.75,1.49,15.82,XY25,NTT,AC
50,MASFIN,398.61,-0.46,-3.23,26.12,22.05,24766.0,-3165.0,94815.0,-16.44,51.0,H-SC,6.88,164.0,-0.13,0.68,37.82,XR,ATH,FINANCE
39,INDIGOPNTS,1408.00,-1.85,-15.01,17.68,0.02,26231.0,-26193.0,148366.0,126.00,45.0,M-SC,14.34,234.0,-1.00,1.07,29.05,OX40N,NTT,PAINTS
43,ITC,452.00,-0.24,-1.78,12.79,10.78,30190.0,-4276.0,236042.0,-41.56,44.0,X-LC,1.99,5.0,-0.14,1.70,3.36,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SFL,1287.00,2.83,-42.72,121.08,26.63,181856.0,-112023.0,150195.0,13.84,36.0,M-SC,11.80,239.0,-0.62,1.08,2.83,XY24,NTT,MISC
16,CERA,9475.00,2.51,-28.26,72.68,23.89,91723.0,-49702.0,126201.0,-30.79,48.0,H-SC,10.70,157.0,-0.54,0.91,8.99,OX40N,NTT,CERAMICS
13,BSOFT,831.70,1.24,-21.79,92.79,50.79,102075.0,-30646.0,110007.0,0.50,65.0,H-SC,6.44,151.0,-0.30,0.79,28.08,XR,ATH,IT
57,RAJOOENG,143.10,0.97,-35.09,115.09,39.61,76569.0,-35970.0,66530.0,-56.03,36.0,H-SC,29.34,135.0,-0.47,0.48,4.82,AR,ATH,MISC
67,SONACOMS,806.63,0.94,-15.32,64.79,39.54,55500.0,-15497.0,85662.0,-33.83,50.0,M-SC,4.42,220.0,-0.28,0.62,20.79,AR,ATH,AUTO


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COFFEEDAY,80.00,-2.14,-42.05,133.44,35.27,87801.0,-47751.0,65798.0,-55.73,44.0,L-SC,10.66,270.0,-0.54,0.47,61.04,XR,NTT,HOTELS
22,DIXON,18931.72,-2.11,-11.17,41.92,26.07,72698.0,-21801.0,173420.0,-56.81,38.0,X-MC,9.65,56.0,-0.30,1.25,8.01,X40N,ATH,IT
71,TANLA,1943.92,-2.10,-37.04,248.94,119.69,424375.0,-100288.0,170473.0,-36.34,42.0,H-SC,7.96,156.0,-0.24,1.23,34.67,AR,ATH,IT
19,COLPAL,3726.84,-1.89,-19.52,75.83,41.51,160729.0,-51405.0,211960.0,-4.10,39.0,X-MC,7.32,61.0,-0.32,1.52,0.84,XY25,ATH,FMCG
39,INDIGOPNTS,1408.00,-1.85,-15.01,17.68,0.02,26231.0,-26193.0,148366.0,126.00,45.0,M-SC,14.34,234.0,-1.00,1.07,29.05,OX40N,NTT,PAINTS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-1.14,-2.38,112.98,107.91,162838.0,-3514.0,144130.0,-20.55,55.0,M-SC,12.55,216.0,-0.02,1.04,7.47,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-1.85,-15.01,17.68,0.02,26231.0,-26193.0,148366.0,126.00,45.0,M-SC,14.34,234.0,-1.00,1.07,29.05,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-0.50,-18.49,22.80,0.09,18729.0,-18633.0,82144.0,90.61,33.0,M-SC,2.47,253.0,-0.99,0.59,7.92,OX40N,NTT,DURABLES
66,SIS,528.00,-0.05,-24.21,60.00,21.26,50490.0,-26882.0,84150.0,1984.97,50.0,H-SC,5.06,166.0,-0.53,0.60,13.70,OX40N,NTT,MISC
47,KANSAINER,340.00,-0.86,-25.71,52.75,13.47,105675.0,-69336.0,200331.0,-69.48,33.0,H-SC,5.67,158.0,-0.66,1.44,1.86,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,0.58,6.94,61.09,72.27,110699.0,11759.0,181207.0,-8.35,65.0,M-LC,2.6,99.0,0.11,1.3,13.54,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.58,6.94,61.09,72.27,110699.0,11759.0,181207.0,-8.35,65.0,M-LC,2.60,99.0,0.11,1.30,13.54,XR,NTT,IT
25,FINCABLES,1641.55,-1.14,-2.38,112.98,107.91,162838.0,-3514.0,144130.0,-20.55,55.0,M-SC,12.55,216.0,-0.02,1.04,7.47,OX40N,ATH,CABLES
38,INDIAMART,4810.62,0.39,-4.17,115.72,106.72,136769.0,-5146.0,118190.0,-52.54,35.0,H-SC,3.29,139.0,-0.04,0.85,18.23,AR,ATH,MISC
85,ZYDUSLIFE,1286.17,0.37,-3.22,40.14,35.63,81413.0,-6751.0,202823.0,-16.74,38.0,H-MC,2.85,119.0,-0.08,1.46,12.91,AR,ATH,PHARMA
37,IEX,219.00,0.58,-4.16,55.08,48.63,105553.0,-8318.0,191636.0,-36.24,48.0,H-SC,14.07,136.0,-0.08,1.38,6.91,XR,NTT,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-0.49,-18.65,91.11,55.47,180322.0,-45369.0,197917.0,-27.82,24.0,X-MC,13.41,64.0,-0.25,1.42,1.41,X40N,ATH,FINANCE
53,PAGEIND,51605.08,-0.70,-10.38,42.95,28.11,77525.0,-20910.0,180500.0,-34.99,26.0,X-MC,13.20,55.0,-0.27,1.30,0.00,X40,ATH,APPARELS
11,BATAINDIA,2096.00,-1.07,-39.45,123.28,35.20,96055.0,-50754.0,77916.0,2.09,29.0,X-SC,16.29,92.0,-0.53,0.56,0.00,X40,NTT,FOOTWEAR
8,AWL,485.00,-0.88,-23.74,101.29,53.50,233075.0,-71644.0,230107.0,-63.12,29.0,X-MC,3.28,58.0,-0.31,1.65,2.21,XY24,NTT,FMCG
33,HINDUNILVR,2922.00,-0.16,-11.40,28.30,13.67,64453.0,-29313.0,227750.0,-18.11,31.0,X-LC,2.30,14.0,-0.45,1.64,7.52,XY25,NTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,CAMS,950.0,-0.76,-1.98,26.99,24.48,66631.0,-4983.0,246873.0,-81.32,42.0,X-SC,0.26,86.0,-0.07,1.77,20.70,X40N,NTT,MISC
43,ITC,452.0,-0.24,-1.78,12.79,10.78,30190.0,-4276.0,236042.0,-41.56,44.0,X-LC,1.99,5.0,-0.14,1.70,3.36,X40,NTT,FMCG
20,DABUR,735.0,-0.12,-2.91,48.01,43.70,115633.0,-7217.0,240851.0,-10.89,37.0,X-MC,2.28,72.0,-0.06,1.73,11.70,XY24,BTT,FMCG
33,HINDUNILVR,2922.0,-0.16,-11.40,28.30,13.67,64453.0,-29313.0,227750.0,-18.11,31.0,X-LC,2.30,14.0,-0.45,1.64,7.52,XY25,NTT,FMCG
12,BERGEPAINT,680.0,-0.50,-4.27,26.55,21.14,57780.0,-9712.0,217627.0,-11.03,39.0,X-MC,2.39,74.0,-0.17,1.56,22.67,XY24,NTT,PAINTS


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.00,-0.13,-25.65,120.79,64.15,213690.0,-61041.0,176910.0,-56.31,33.0,X-SC,2.78,82.0,-0.29,1.27,0.00,XY24,BTT,CEMENT
11,BATAINDIA,2096.00,-1.07,-39.45,123.28,35.20,96055.0,-50754.0,77916.0,2.09,29.0,X-SC,16.29,92.0,-0.53,0.56,0.00,X40,NTT,FOOTWEAR
53,PAGEIND,51605.08,-0.70,-10.38,42.95,28.11,77525.0,-20910.0,180500.0,-34.99,26.0,X-MC,13.20,55.0,-0.27,1.30,0.00,X40,ATH,APPARELS
31,HAVELLS,2069.16,-0.95,-10.45,47.95,32.48,140832.0,-34285.0,293706.0,-15.05,39.0,X-MC,6.10,67.0,-0.24,2.11,0.63,X40,ATH,ELECTRICAL
59,RELAXO,1176.00,-0.16,-48.15,193.74,52.31,145820.0,-69894.0,75266.0,-44.83,39.0,X-SC,7.10,91.0,-0.48,0.54,0.75,X40N,NTT,FOOTWEAR


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,-1.31,-30.31,104.96,42.84,47552.0,-19701.0,45305.0,-54.27,40.0,X-SC,35.37,83.0,-0.41,0.33,2.50,XY24,NTT,MISC
59,RELAXO,1176.00,-0.16,-48.15,193.74,52.31,145820.0,-69894.0,75266.0,-44.83,39.0,X-SC,7.10,91.0,-0.48,0.54,0.75,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-1.07,-39.45,123.28,35.20,96055.0,-50754.0,77916.0,2.09,29.0,X-SC,16.29,92.0,-0.53,0.56,0.00,X40,NTT,FOOTWEAR
51,MEDANTA,1486.00,-0.53,-5.32,31.05,24.08,37320.0,-6757.0,120193.0,-13.03,31.0,X-SC,5.18,89.0,-0.18,0.86,13.25,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,-0.96,-17.27,73.24,43.32,98684.0,-28132.0,134740.0,-28.92,34.0,X-SC,4.90,90.0,-0.29,0.97,3.46,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,0.62,-11.33,36.13,20.71,121176.0,-42847.0,335390.0,-23.62,62.0,X-LC,4.24,1.0,-0.35,2.41,12.07,X40,ATH,IT
41,INFY,1972.00,0.58,9.06,23.09,34.24,78794.0,28359.0,341247.0,-14.93,61.0,X-LC,4.28,2.0,0.36,2.45,18.10,X40,NTT,IT
75,TMPV,600.00,0.27,-29.00,73.21,22.98,117416.0,-65515.0,160383.0,-26.74,31.0,X-LC,6.75,3.0,-0.56,1.15,0.87,XY24,NTT,AUTO
81,VBL,671.64,0.04,-3.58,40.70,35.67,123951.0,-11293.0,304549.0,-14.99,56.0,X-LC,3.97,4.0,-0.09,2.19,9.78,X40N,ATH,FMCG
43,ITC,452.00,-0.24,-1.78,12.79,10.78,30190.0,-4276.0,236042.0,-41.56,44.0,X-LC,1.99,5.0,-0.14,1.70,3.36,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,0.88,-16.09,116.50,81.67,91709.0,-15090.0,78720.0,7072.22,48.0,L-SC,12.06,271.0,-0.16,0.57,52.45,XR,NTT,CERAMICS
50,MASFIN,398.61,-0.46,-3.23,26.12,22.05,24766.0,-3165.0,94815.0,-16.44,51.0,H-SC,6.88,164.0,-0.13,0.68,37.82,XR,ATH,FINANCE
13,BSOFT,831.70,1.24,-21.79,92.79,50.79,102075.0,-30646.0,110007.0,0.50,65.0,H-SC,6.44,151.0,-0.30,0.79,28.08,XR,ATH,IT
39,INDIGOPNTS,1408.00,-1.85,-15.01,17.68,0.02,26231.0,-26193.0,148366.0,126.00,45.0,M-SC,14.34,234.0,-1.00,1.07,29.05,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,-0.16,-2.61,24.18,20.95,59290.0,-6565.0,245203.0,-3.58,48.0,X-LC,17.12,31.0,-0.11,1.76,32.10,X40,ATH,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.70,1.24,-21.79,92.79,50.79,102075.0,-30646.0,110007.0,0.50,65.0,H-SC,6.44,151.0,-0.30,0.79,28.08,XR,ATH,IT
84,WIPRO,420.00,0.58,6.94,61.09,72.27,110699.0,11759.0,181207.0,-8.35,65.0,M-LC,2.60,99.0,0.11,1.30,13.54,XR,NTT,IT
41,INFY,1972.00,0.58,9.06,23.09,34.24,78794.0,28359.0,341247.0,-14.93,61.0,X-LC,4.28,2.0,0.36,2.45,18.10,X40,NTT,IT
73,TCS,4389.97,0.62,-11.33,36.13,20.71,121176.0,-42847.0,335390.0,-23.62,62.0,X-LC,4.24,1.0,-0.35,2.41,12.07,X40,ATH,IT
27,GLAXO,3466.20,-0.02,3.77,34.09,39.15,70498.0,7516.0,206800.0,-16.74,53.0,X-MC,10.61,60.0,0.11,1.49,33.87,X40,ATH,PHARMA


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.31
1,25,44.86
2,50,76.68


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.95
MC    30.91
LC    24.16
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.70
X40      23.04
X40N     13.04
XR        9.76
AR        9.06
XY25      9.06
OX40N     7.60
SR        0.99
MH        0.77
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.42
H-SC    23.81
X-LC    20.71
M-SC    11.69
X-SC     8.04
H-MC     4.83
L-SC     1.41
M-MC     1.32
M-LC     1.30
H-LC     1.13
L-LC     1.02
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.37,-7.94,42.65
IT,13.04,-15.89,74.16
FINANCE,9.45,-21.18,73.88
MISC,7.30,-27.93,80.26
ELECTRICAL,6.01,-12.26,53.38
PAINTS,5.83,-13.78,30.68
INSURANCE,4.78,-2.12,37.06
PHARMA,4.15,-1.99,34.09
AUTO,2.82,-33.46,79.74


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3206756.0,21
AR,1312950.0,10
XR,1304560.0,13
X40,1019752.0,14
X40N,870508.0,9
OX40N,734948.0,10
XY25,379373.0,6
SR,282785.0,2
MH,77659.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3669176.0,25
M-SC,1433867.0,15
X-MC,1428417.0,16
X-LC,844704.0,11
X-SC,800236.0,8
H-MC,407503.0,3
L-SC,264972.0,3
M-LC,110699.0,1
H-LC,76327.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1286011.0      6
           AR         898621.0      5
M-SC       XY24       822856.0      6
H-SC       XR         784845.0      7
X-MC       X40        497852.0      7
           XY24       406488.0      3
           X40N       341784.0      4
H-SC       OX40N      339255.0      4
X-LC       X40        331361.0      5
X-SC       X40N       311135.0      3
M-SC       OX40N      310231.0      5
X-SC       XY24       298562.0      3
H-SC       SR         282785.0      2
X-LC       X40N       217589.0      2
H-MC       AR         211880.0      2
X-LC       XY24       197216.0      2
H-MC       XY24       195623.0      1
X-SC       X40        190539.0      2
X-MC       XY25       182293.0      2
L-SC       XR         179510.0      2
M-SC       XR         174658.0      2
           AR         126122.0      2
M-LC       XR         110699.0      1
X-LC       XY25        98538.0      2
L-SC       OX40N       85462.0      1
H-SC       MH          77659.0      1
H-LC       AR          76327.0      1
M-MC       XY25        61402.0      1
L-MC       XR          54848.0      1
L-LC       XY25        37140.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 41.0 seconds
